In [1]:
import numpy as np
from numpy import linalg as la
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import pickle

def generate_subspace(D, k):
    
    mean = list()
    for Di in D:
        meani = (Di[0] + Di[1]) / 2
        new = list()
        new.append(meani)
        new.append(meani)
        mean.append(new)

    mean = np.array(mean)
    D_centralize = D - mean
    
    D0 = D[0]
    v0 = D0[0]
    v0 = np.array(v0).reshape((1,-1))
    C = np.zeros((np.matmul(v0.T, v0)).shape)
    for Di in D_centralize:
        v0 = Di[0]
        v1 = Di[1]
        v0 = np.array(v0).reshape((1,-1))
        v1 = np.array(v1).reshape((1,-1))
        Ci = (v0.T.dot(v0) + v1.T.dot(v1)) / 2
        C += Ci

    U, sigma, VT = la.svd(C)
    return U[: k]

In [2]:
def projection_matrix_general(B):
    """Compute the projection matrix onto the space spanned by `B`
    Args:
        B: ndarray of dimension (d, k), the basis for the subspace
        is [b1, b2, ..., bk]
    
    Returns:
        P: the projection matrix
    """
    P = B.dot(la.pinv(B.T.dot(B))).dot(B.T)
    #P = np.dot(np.dot(B, la.pinv(B.T, B)), B.T)
    return P

In [3]:
def project_general(x, B):
    """Compute the projection matrix onto the space spanned by `B`
    Args:
        B: ndarray of dimension (k, d), the basis for the subspace
    
    Returns:
        y: projection of x in space spanned by b
    """
    p = np.zeros(x.shape)
    P = np.dot(projection_matrix_general(B.T), x)
    return p

In [4]:
def neutralize(B, w):
    wb = project_general(w, B)
    wp = w - wb
    return wp

In [5]:
def findStrInFile(str):
    f_pro = open('professions.txt')
    for line in f_pro.readlines():
        l = len(line)
        if str == line[:l - 1]:
            f_pro.close()
            return True
    f_pro.close()
    return False

In [6]:
import string
import re

# 对于 model 中训练得到的词向量，依次查看词是否为职业词，若是，则做中性化处理

fout = open('wiki.zh.debias.txt', 'w')
model = word2vec.Word2Vec.load('wiki.zh.model')

D = list()
W = ["他", "她", "他们", "她们", "男性", "女性", "父亲", "母亲", "儿子", "女儿", 
     "爸爸", "妈妈", "男孩", "女孩", "男生", "女生", "男人", "女人", "男子", "女子"]
i = 0
while i < len(W):
    Di = list()
    Di.append(model.wv[W[i]])
    Di.append(model.wv[W[i + 1]])
    D.append(Di)
    i += 2

# 现在 D 中存有 10 * 2 * 100 个元素：
# 10 个定义集，每个定义集中有 2 个性别对立的词向量，每个词向量有 300 个维度

D = np.array(D)
B = generate_subspace(D, 10)

for word in model.wv.vocab.keys():
    if findStrInFile(word):
        w = model.wv[word]
        w_n = neutralize(B, w)
        s = ""
        s += word + " "
        for i in w_n:
            s += str(i) + " "
        s += "\n"
        fout.write(s)
    else:
        w = model.wv[word]
        s = ""
        s += word + " "
        for i in w:
            s += str(i) + " "
        s += "\n"
        fout.write(s)

fout.close()